In [2]:
import numpy as np
import mne
import os
import pandas as pd
import pywt
import torch
from datetime import datetime
from scipy.signal import iirfilter, sosfilt


from mne.preprocessing import ICA
from mne.preprocessing import create_eog_epochs
from mne.preprocessing import read_ica

In [3]:
def get_epochs_and_labels(fif_file):
    # Load your EEG data
    raw = mne.io.read_raw_fif(fif_file, preload=True)
    
    # Get the events from the annotations
    events, _ = mne.events_from_annotations(raw)

    # T1 is left hand, T2 is right hand
    event_id = {'T1': 2, 'T2': 3}

    # Epochs start 0s before the trigger and end 0.5s after
    epochs = mne.Epochs(raw, events, event_id, tmin=0, tmax=0.5, baseline=None, preload=True)

    # Get the labels of the epochs
    labels = epochs.events[:, -1]

    # Change the labels to 5 and 6
    labels[labels == 2] = 5
    labels[labels == 3] = 6
    
    return epochs, labels

In [4]:
def z_score(epoch):    
    # Apply z-score normalization to each channel, saved in epoch
    for i in range(epoch.shape[0]):
        channel_epoch = epoch[i, :]
        mean = np.mean(channel_epoch)
        std = np.std(channel_epoch)
        z_scored_epoch = (channel_epoch - mean) / std
        epoch[i, :] = z_scored_epoch
    
    return epoch

In [5]:
# Function to convert frequency to wavelet scale
def frequency_to_scale(freq, wavelet='cmor', sampling_rate=250):
    center_freq = pywt.central_frequency(wavelet)
    return center_freq / (freq / sampling_rate)

# Function to apply wavelet transform to EEG data for a given frequency band
def apply_wavelet_transform_band(data_norm, wavelet, freq_range, sampling_rate=250):
    n_channels, n_times = data_norm.shape
    scales = frequency_to_scale(np.arange(freq_range[0], freq_range[1] + 1), wavelet=wavelet, sampling_rate=sampling_rate)
    
    coeffs = []
    for i in range(n_channels):
        coef, _ = pywt.cwt(data_norm[i], scales=scales, wavelet=wavelet)
        coeffs.append(coef)
    
    coeffs_done = np.stack(coeffs, axis=0)
    return coeffs_done

# IIR Bandpass Filter function
def bandpass_filter(data, lowcut, highcut, sampling_rate=250, order=5):
    sos = iirfilter(order, [lowcut, highcut], btype='band', ftype='butter', fs=sampling_rate, output='sos')
    return sosfilt(sos, data)

# Function to apply wavelet transform to multiple frequency bands and combine the results
def apply_wavelet_transform(data_norm, wavelet='cmor', sampling_rate=250):
    # Define frequency ranges for alpha, beta, and gamma bands
    alpha_range = (8, 13)
    beta_range = (14, 30)
    gamma_range = (31, 50)
    
    # Apply bandpass filter to each frequency band
    data_alpha = bandpass_filter(data_norm, alpha_range[0], alpha_range[1], sampling_rate=sampling_rate)
    data_beta = bandpass_filter(data_norm, beta_range[0], beta_range[1], sampling_rate=sampling_rate)
    data_gamma = bandpass_filter(data_norm, gamma_range[0], gamma_range[1], sampling_rate=sampling_rate)
    
    # Apply wavelet transform to each frequency band
    coeffs_alpha = apply_wavelet_transform_band(data_alpha, wavelet, alpha_range, sampling_rate)
    coeffs_beta = apply_wavelet_transform_band(data_beta, wavelet, beta_range, sampling_rate)
    coeffs_gamma = apply_wavelet_transform_band(data_gamma, wavelet, gamma_range, sampling_rate)
    
    # Concatenate the wavelet coefficients along the scale dimension
    coeffs_combined = np.concatenate((coeffs_alpha, coeffs_beta, coeffs_gamma), axis=1)
    
    return coeffs_combined

# Example usage
# data_norm is the normalized EEG data with shape (channels, time_points)
# data_wavelet = apply_wavelet_transform(data_norm, wavelet='cmor', sampling_rate=250)


In [6]:
def file_to_tensor(fif_file):
    epochs, labels = get_epochs_and_labels(fif_file)
    
    # Create an empty list to store the transformed epochs
    transformed_epochs = []
    
    # Z-score normalization and wavelet transform for each epoch and stack them into a tensor (4D array), 
    # shape (n_epochs, n_channels, n_scales, n_time_points)
    # Loop through each epoch
    for epoch in epochs:
        # Z-score each epoch
        epoch_norm = z_score(epoch)
        
        # Apply wavelet transformation
        wavelet_tensor = apply_wavelet_transform(epoch_norm)
        
        # Append the transformed epoch to the list
        transformed_epochs.append(wavelet_tensor)

    # Convert the list of epochs into a tensor dataset
    tensor_dataset = torch.tensor(transformed_epochs)
            
    return tensor_dataset

In [7]:
def folder_to_tensor(folder_path, output_folder):
    """
    Combine multiple EEG files into a single PyTorch tensor dataset.
    
    Parameters:
    folder_path (str): Path to the folder containing the EEG files
    output_folder (str): Path to the folder where the combined dataset will be saved
    
    Returns:
    torch.Tensor: Combined EEG dataset as a PyTorch tensor
    
    """
    # Create an empty list to store tensors from each file
    all_tensors = []
    
    # Iterate through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".fif"):
            file_path = os.path.join(folder_path, filename)
            tensor = file_to_tensor(file_path)  # Assuming tensor_from_file is defined
            all_tensors.append(tensor)
    
    # Concatenate all tensors along the first dimension to create one dataset
    combined_dataset = torch.cat(all_tensors, dim=0)
    
    # Get the current date and time
    current_datetime = datetime.now()
    
    # Format the date and time as a string for the dataset name
    dataset_name = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    
    # Save the combined dataset to the output folder with the specified name
    output_file = os.path.join(output_folder, f"{dataset_name}.pt")
    torch.save(combined_dataset, output_file)
    
    return combined_dataset

In [9]:
input_folder = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\external\interim"
output_folder = r"C:\School\EE _Y3\Q4\BAP\eeg_thesis_cnn_repo\data\external\tensors"

tensor_dataset_total = folder_to_tensor(input_folder, output_folder)

Opening raw data file C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\external\interim\S001R03_filtered.fif...
Isotrak not found
    Range : 0 ... 31249 =      0.000 ...   124.996 secs
Ready.
Reading 0 ... 31249  =      0.000 ...   124.996 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
15 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 15 events and 126 original time points ...
0 bad epochs dropped


C:\Users\chakr\AppData\Local\Temp\ipykernel_13788\3980180034.py:3: RuntimeWarning: This filename (C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\external\interim\S001R03_filtered.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fif_file, preload=True)
c:\Users\chakr\Anaconda3\envs\eeg_thesis\lib\site-packages\pywt\_functions.py:143: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should takethe form cmorB-C where B and C are floats representing the bandwidth frequency and center frequency, respectively (example: cmor1.5-1.0).
  wavelet = DiscreteContinuousWavelet(wavelet)
c:\Users\chakr\Anaconda3\envs\eeg_thesis\lib\site-packages\pywt\_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters spe

Opening raw data file C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\external\interim\S001R07_filtered.fif...
Isotrak not found
    Range : 0 ... 31249 =      0.000 ...   124.996 secs
Ready.
Reading 0 ... 31249  =      0.000 ...   124.996 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
15 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 15 events and 126 original time points ...
0 bad epochs dropped


C:\Users\chakr\AppData\Local\Temp\ipykernel_13788\3804080640.py:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:277.)
  tensor_dataset = torch.tensor(transformed_epochs)
C:\Users\chakr\AppData\Local\Temp\ipykernel_13788\3980180034.py:3: RuntimeWarning: This filename (C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\external\interim\S001R07_filtered.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fif_file, preload=True)
c:\Users\chakr\Anaconda3\envs\eeg_thesis\lib\site-packages\pywt\_functions.py:143: FutureWarning:

Opening raw data file C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\external\interim\S001R11_filtered.fif...
Isotrak not found
    Range : 0 ... 31249 =      0.000 ...   124.996 secs
Ready.
Reading 0 ... 31249  =      0.000 ...   124.996 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
15 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 15 events and 126 original time points ...
0 bad epochs dropped


...

Opening raw data file C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\external\interim\S109R07_filtered.fif...
Isotrak not found
    Range : 0 ... 30749 =      0.000 ...   122.996 secs
Ready.
Reading 0 ... 30749  =      0.000 ...   122.996 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
15 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 15 events and 126 original time points ...
0 bad epochs dropped


C:\Users\chakr\AppData\Local\Temp\ipykernel_13788\3980180034.py:3: RuntimeWarning: This filename (C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\external\interim\S109R07_filtered.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fif_file, preload=True)
c:\Users\chakr\Anaconda3\envs\eeg_thesis\lib\site-packages\pywt\_functions.py:143: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should takethe form cmorB-C where B and C are floats representing the bandwidth frequency and center frequency, respectively (example: cmor1.5-1.0).
  wavelet = DiscreteContinuousWavelet(wavelet)
c:\Users\chakr\Anaconda3\envs\eeg_thesis\lib\site-packages\pywt\_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters spe

Opening raw data file C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\external\interim\S109R11_filtered.fif...
Isotrak not found
    Range : 0 ... 30749 =      0.000 ...   122.996 secs
Ready.
Reading 0 ... 30749  =      0.000 ...   122.996 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
15 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 15 events and 126 original time points ...
0 bad epochs dropped


C:\Users\chakr\AppData\Local\Temp\ipykernel_13788\3980180034.py:3: RuntimeWarning: This filename (C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\external\interim\S109R11_filtered.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fif_file, preload=True)
c:\Users\chakr\Anaconda3\envs\eeg_thesis\lib\site-packages\pywt\_functions.py:143: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should takethe form cmorB-C where B and C are floats representing the bandwidth frequency and center frequency, respectively (example: cmor1.5-1.0).
  wavelet = DiscreteContinuousWavelet(wavelet)
c:\Users\chakr\Anaconda3\envs\eeg_thesis\lib\site-packages\pywt\_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters spe

RuntimeError: Parent directory C:\School\EE _Y3\Q4\BAP\eeg_thesis_cnn_repo\data\external\tensors does not exist.